# Lab 01 - Understanding the Workflow

Since ConfigSync file structure is compartmentalized it is important to understand how those components work when 
developing new code.  To recap, all interactions that function with Cisco Viptela Vmanage should be contained within the viptela.py file, all methods to deal with data retrieved from the vManage API should go in helpers.py, and tie it all together in config_sync.py. If you need to add configuration data, it can be added into the viptela.cfg file.

To recreate how this functions, we will build the basic application within the Jupyter Lab Notebook below.  In order for this code to execute properly, you must go into each code snippet and click the Play button in the toolbar.  The example we will rebuild is the first use case, which is pulling a list of Devices from vManage.

## Base Configuration
The base configuration we will start with logging, argparse, base classes for both ConfigSync(config_sync.py) and ViptelaClient (viptela.py).  The login function has already been defined, and called when ConfigSync is initialized.  Everything else will be added in the code segments below.

The files in the directory /Labs/lab01 have been set to a base configuration state described above.

**config_sync.py**

```python
from viptela import ViptelaClient
import argparse
import logging
import yaml
import helpers


def parse_arguments():
    parser = argparse.ArgumentParser()
    parser.add_argument("--config", "-c", help="Path to the configuration file", default="viptela.cfg")
    parser.add_argument("--debug", "-d", help="Display debug logs", action="store_true")
    return parser.parse_args()


def init_logger(log_level=logging.INFO):
    logger = logging.getLogger(__file__)
    logger.setLevel(log_level)

    console_handler = logging.StreamHandler()
    formatter = logging.Formatter('%(asctime)s %(levelname)s: %(message)s')
    console_handler.setFormatter(formatter)
    logger.addHandler(console_handler)
    if log_level == 10:
        # create file handler which logs even debug messages
        fh = logging.FileHandler('log_file.txt')
        formatter = logging.Formatter('%(asctime)s %(levelname)s: %(message)s')
        fh.setFormatter(formatter)
        fh.setLevel(logging.DEBUG)
        logger.addHandler(fh)
    return logger


class ConfigSync:

    def __init__(self, config, log):
        self.log = log
        self.config_file = config
        self.log.info('Initializing ConfigSync Class.')
        self.config = self._parse_config(config)
        self.viptela = self._init_viptela_client(self.config)
        self.log.debug('Finished initializing the configSync class.')

    def _parse_config(self, config_file):
        self.log.info('Parsing the configuration file.')
        with open(config_file, 'r') as f:
            config = yaml.safe_load(f)
        self.log.debug(f'The following parameters were received: {config}')
        return config

    def _init_viptela_client(self, config):
        self.log.info('Initializing ViptelaClient class.')
        host = config.get('viptela_host')
        username = config.get('viptela_username')
        password = config.get('viptela_password')
        port = config.get('viptela_port')
        self.log.debug(f'Username is {username} and password is {password}')
        viptela = ViptelaClient(host, username=username, password=password, port=port, log=self.log)
        self.log.info('Login to Viptela.')
        viptela.login_sdk()
        return viptela


if __name__ == "__main__":
    args = parse_arguments()

    if args.debug:
        log = init_logger(logging.DEBUG)
    else:
        log = init_logger()

    log.info(f'Viptela ConfigSync script has started.')
    # Instantiate ConfigSync which will launch connection to Viptela, setup logging, and
    # load configuration from config file.
    cs = ConfigSync(config=args.config, log=log)
    # Start custom scripting here.  Use the CS object to interact with functions above or pull modules
    # directly from the helper file.

```


**viptela.py**

```python
import requests
from vmanage.api.authentication import Authentication


class ViptelaClient:

    def __init__(self, host, port=443, username='admin', password='admin', log=None):
        self.host = host
        self.port = port
        self.username = username
        self.password = password
        self.log = log
        if not log:
            raise Exception('The logger should not be None.')

        self.cookie = None
        self.session = None
        self.base_headers = {
            'Content-Type': 'application/json',
            'Accept': 'application/json',
        }
        self.base_url = f'https://{self.host}:{self.port}/'

        requests.packages.urllib3.disable_warnings()

        self.log.debug('ViptelaClient class initialization finished.')

    def login_sdk(self):
        self.log.debug(f'Login to Viptela vManage with Python SDK at host {self.host}.')
        self.session = Authentication(host=self.host, user=self.username, password=self.password,
                                 port=self.port).login()
        self.cookie = self.session.cookies._cookies[self.host]["/"]["JSESSIONID"]
        self.log.debug(f'Viptela provided authentication cookie: {self.session.cookies._cookies[self.host]["/"]["JSESSIONID"]}.')
        

```

**helpers.py** - File is empty because we haven't created any helpers yet.

```
<EMPTY>
```

**viptela.cfg*** - Default Config
```
---
viptela_host: '198.18.133.200'
viptela_username: 'admin'
viptela_password: 'admin'
viptela_port: 8443

```

## Run the code

Let's run the code and see what happens.  Click in the box below, then click the play button &#9654; or Shift+Enter to launch the script.

In [ ]:
%run config_sync.py --debug

## Review output

Look at the log file displayed above. The script started, initialized the ConfigSync class, parsed the configuration 
file to pull login parameters, Initialized ViptelaClient, Logged into vManage and retrieved cookies which will be used in future interaction during this script.

## More detail

Let's look in more detail at what is going on in the config_sync.py file.  Below is the snipped of code at the bottom config_sync.py that launches **everything** that happens in this script.

**config_sync.py**
```python
.
.
. <INFO OMITTED FOR BREVITY>
.
if __name__ == "__main__":
    args = parse_arguments()

    if args.debug:
        log = init_logger(logging.DEBUG)
    else:
        log = init_logger()

    log.info(f'Viptela ConfigSync script has started.')
    # Instantiate ConfigSync which will launch connection to Viptela, setup logging, and
    # load configuration from config file.
    cs = ConfigSync(config=args.config, log=log)
    # Start custom scripting here.  Use the CS object to interact with functions above or pull modules
    # directly from the helper file.
```

# Review the steps

1. The action portion of the script start this standard best practice which tells the python interpretter that if this file is launched from a command line (which inherrently launches any functions called 'main'), to then run commands below.  This command is always put at the bottom of the file and helps concentrate all the actions into one place.  
```python
if __name__ == "__main__"
```

2. Launch the parse_arguments function within this file and stores the output as args.
```python
    args = parse_arguments()
```

3. Determines if the --debug CLI flag was set, and if so initializes the logger and sets the level to DEBUG (10).  Variable **log** is now how we interact with the logging mechanism.

```python
    if args.debug:
        log = init_logger(logging.DEBUG)
    else:
        log = init_logger()
```

4.  Log a record that the ViptelaClient class is being instantiated.  This action passes all config file variables as well as the logger created above.  Both of these are required to instantiate the class which will result in either vManage being logged in or an error will have occured.
```python
    log.info(f'Viptela module ViptelaClient is starting.')
    # Instantiate ConfigSync which will launch connection to Viptela, setup logging, and
    # load configuration from config file.
    cs = ConfigSync(config=args.config, log=log)
    # Start custom scripting here.  Use the CS object to interact with functions above or pull modules
    # directly from the helper file.
```

That's all there is to getting started here.  Now let's move on to add some functionality to the script.

## Discover all devices in vManage and print output to the console
For this first exercise you will be adding one of the workflows included in the framework, gathering all devices from the vManage orchestrator. The workflow we will be creating is to create both a viptela module that will pull all devices, and a helper module that will take that device info and parse it into a table to display on CLI.  The viptela function will be housed within the ViptelaClient class within viptela.py, and the print table module will be in the helpers.py file.  You will then add the instructions to config_sync to tell it to gather the data from the new Viptela class, and then pass that data to the new helper class.

### Step 1 - Open Pycharm on Desktop

### Step 2 - Open ViptelaClient project

### Step 3 - Open file /Labs/lab01/config_sync.py, /Labs/lab01/viptela.py, and /Labs/lab01/helpers.py in Pycharm tabs

### Step 4 - Review API Reference for the Viptela SDK

https://python-viptela.readthedocs.io/en/latest/

You want to interact with the Devices module, so click on the api link and then choose api.device.  Two things to note here are the class details highlighted in blue, and the functions wihhch are bolded in grey boxes below.  The very first module, get_device_list(self, category), is the module you will be interacting with with.  We must instantiate a copy of this class using the parameters required in the blue box and then call the appropriate function, get_device_list for the results.  The parameters that it accepts are either vedges or controllers, and it will return our data as a dictionary.

### Step 5 - viptela.py - import the 'Devices' module 

To use this piece of the Python SDK, we first have to import the the module into our viptela.py file.  Add the following as the last line in the import statements at the top of the file.

```python
from vmanage.api.device import Device
```
### Step 6 - viptela.py - create new module in ViptelaClient class

Once the module is imported, we can begin to interact with the module within our ViptelaClass.  Go to the viptela.py file, and scroll to the bottom of the file, which should be right below 'def login_sdk(self):'.  We are going to add our new function here which will be called form within config_sync.py. Copy the code snippet below into the file in Pycharm.

The function within viptela.py should do three things.

1. Instantiate the Device class we just imported using settings we already have because we instantiated ViptelaClient.

2. Send the 'type' of device we get (either vedges or controllers) to the get_device_list function in the Viptela SDK and save that data to a variable called device_list

3. Return device_list

```python
    def get_devices_list(self, type):
        '''
        Get devices uses the Viptela Device API to fetch the list of devices
        either vedges or controllers and returns them in a list of dictionaries.

        :param type: Either 'vedges' or 'controllers'
        :return:
            result (list): Device list
        '''

        # Instantiate new device object to use the Device library
        device = Device(self.session, self.host, port=self.port)
        # Request list of all devices by type specified and store in variable
        device_list = device.get_device_list(type)
        return device_list
```

### Step 7 - config_sync.py - Call the new viptela function

The function has been defined in viptela.py but we need to update config_sync.py to launch that function.  To keep code as readable and reusable as possible we are going to split this into two tasks.  First, create a new function under ConfigSync class that will bundle all of the requests together and return the data.  Second, call that function from under the "if __name__ == '__main__'" section of the script.  To avoid any confusion between the function name in ConfigSync and the function name in ViptelaClient, prepend all ConfigSync functions with cs_.

1. Create the function cs_get_all_device_info and gather info for both vedges and controllers.  Return the data.

```python
    def cs_get_all_device_info(self):
        vedges = self.viptela.get_devices_list('vedges')
        controllers = self.viptela.get_devices_list('controllers')
        return vedges, controllers
```

2. Call that function from under the "if __name__ == '__main__'".  With this one call, store both vedge and controller data when it is returned.  We will use these list independently to perform different functions.

```python
    # Get Device info for vedges and controllers
    vedge_list, controller_list = cs.cs_get_all_device_info()
```

### Step 8 - Pycharm - Debug the code

Let's see what happens when we launch the script and step through it.  If you were to run the code fully, it would process but not shot any output.  To debug the code we have to set a break point, and then step through the code using either F8 (next step) or F7 (step into).  If we get in too deep in our steps, we can step out with Shift + F8.  

As you are debugging, pay attention to the Debugger window at the bottom.  It will contain all variables currently in memory at that step.  Debugging is a great way to understand exactly what data and in what formats are we receiving from the API.

1. Set a breakpoint

Set breakpoint in the last line by clicking on the line number.  A red dot should appear beside it as shown below.

![Set Breakpoint](images/lab01-001-debug_breakpoint_1.jpg)

2. Launch Labs/lab01/config_sync.py from within Pycharm in DEBUG mode

Right click on the file in the Project Explorer and choose 'DEBUG config_sync.py'.  This will launch us into DEBUG mode, start executing the application, and stop when it hits our DEBUG line.

3.  Step INTO our functions

From the breakpoint if we hit F7 it will step INTO that line, meaning it will go to the function/module that is being called.  Go ahead and hit F7 and it should take us to 'def cs_get_all_device_info'.  At this point nothing has executed in this function, we have just been moved into it.  Now press F7 again to step INTO this function.  It will take us to Labs/lab01/viptela.py at the function 'def get_devices_list'.  Again nothing has executed yet, we are just sitting at the first line.

4.  Step INTO the vManage SDK

To sneak a little farther under the hood, go ahead and press F7 one more time here.  This will take us to the devices.py file, which is part of the vManage SDK. Notice that we are in the __init__ function which is automatic when instantiating a new class.  Go ahead and step through here with F8 - 3 times.  Notice as you press F8 a grey variable pops up to the right of each line.  This is how Pycharm shows us where a variable has been assigned, and what the value is.  We could also look in the variables list below and see these details.  NOTE:  Variables change as we step through programs.  

If you press F8 one more time, it will finish the last step in __init__ and send us back to our function in viptela.py.  

5. Step INTO the vManage SDK again

Press F8 once more and it should move us down to the function call 'device_list = device.get_device_list(type)'.  Press F7 on this line to step INTO the devices.py once more.

Notice we are now in the 'def get_device_list' function within devices.py.  We have passed a variable for category with our value being 'vedges'.  Notice how simple and clean even this portion of the SDK is.  We construct a URL, make a REST call, parse the data, and then return it to the function that called.  

Go ahead and step OVER three (3) of these events by pressing F8.  This should leave you on the line 'return results'.  Take a look at 'response' and 'result' in the DEBUGGER window at the bottom of your screen.  NOTE:  The data included in results can be found in the REST response under response{'json'}{'data'}.  Reviewing this data tells us exactly what type of format the return results are in, as well as allows us to dig into those results to understand what variables, values, etc are in use.

6.  Follow the data results

Step OUT by pressing F8 one more time.  This will complete the function and return the results. Press F8 once more to step OVER and to the final line which is 'return device_list'.  Note how the same data in the response is now in the device_list variable.

Press F8 again to step OVER and finish this function.

7.  Step OUT and finish script

Feel free to step around in this application as much as you want to, but it is not necessary for us to finish what we are doing. 

To step OUT press Shift + F8 **twice** and the program will complete.  Note that nothing else happens because we haven't done anything with the data yet.

### Step 9 - helpers.py - Import necessary modules to helpers.py

In order to format the output of the results we are going to use a module called 'tabulate'.  Import this module into the helpers.py file at the top of the file.

```python
import tabulate
```

### Step 10 - helpers.py - Create new function in helpers.py to print the device data to screen

Add the following code snippet into helpers.py.  The function will take the input devices_list, create a list for headers based on the fields we will be pulling out of the data set.  To populate the table, iterate through the devices list passed over, and for each device gather the fields specified in the headers.  

Once the table is populated with all devices, we pass both the table and headers into the tabulate module.  We use try/except blocks as error handling.  In the event there is a unicode error, a modified version of this grid will be populated.

```python
def print_device_table(devices):

    headers = [
        'Host-Name',
        'Device Type',
        'Device ID',
        'Serial Number',
        'System IP',
        'Site ID',
        'Version',
        'Device Model',
        'Template',
    ]

    table = []

    for device in devices:
        if 'host-name' in device and 'system-ip' in device:
            if not 'template' in device:
                device['template'] = ""
            row = [
                device['host-name'],
                device['deviceType'],
                device['uuid'],
                device['serialNumber'],
                device['system-ip'],
                device['site-id'],
                device['version'],
                device['deviceModel'],
                device['template'],
            ]
            table.append(row)

    try:
        print(tabulate.tabulate(table, headers, tablefmt="fancy_grid"))
    except UnicodeEncodeError:
        print(tabulate.tabulate(table, headers, tablefmt="grid"))
```

### Step 11 - config_sync.py - Add helpers.print_device_table to print the device list

At the bottom of the 'if __main__ == "__main__"' section add the following lines.  This will print both the vedge list and the controller list received in the line above.

```python
    helpers.print_device_table(vedge_list)
    helpers.print_device_table(controller_list)
```


### Step 12 - Save & launch the code again

Now we should have everything needed to launch this function.  Go ahead and press the Run button to execute the script in Pycharm.  We should have the log messages displayed, and then two table printed out with vedge and controller info.  You can also run the code in the box below.


%run config_sync.py

### NOTE: If you have any issues compare to the files in the /final folder.